In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=231):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "075580"
    company_name = "세진중공업"

    # 데이터 수집
    df = get_day_price(code, max_pages=231)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜     종가  전일비     시가     고가     저가     거래량
2025.04.25 8780.0  100 8890.0 8970.0 8640.0  556001
2025.04.24 8680.0  370 8310.0 8710.0 8140.0  699050
2025.04.23 8310.0  -10 8410.0 8410.0 8270.0  109595
2025.04.22 8320.0   30 8190.0 8430.0 8180.0  211845
2025.04.21 8290.0 -190 8490.0 8530.0 8110.0  390526
2025.04.18 8480.0  -20 8630.0 8710.0 8330.0  336868
2025.04.17 8500.0   70 8550.0 8610.0 8370.0  306009
2025.04.16 8430.0 -170 8500.0 8580.0 8310.0  354327
2025.04.15 8600.0  -20 8640.0 8670.0 8400.0  460028
2025.04.14 8620.0  730 8270.0 8650.0 8090.0 1182855
2025.04.11 7890.0  200 7800.0 8050.0 7750.0 760177
2025.04.10 7690.0  690 7360.0 7720.0 7210.0 628467
2025.04.09 7000.0  150 6950.0 7120.0 6820.0 357940
2025.04.08 6850.0  430 6600.0 6990.0 6530.0 388659
2025.04.07 6420.0 -490 6680.0 6740.0 6410.0 317220
2025.04.04 6910.0   70 6790.0 7130.0 6760.0 250580
2025.04.03 6840.0   40 6660.0 6880.0 6610.0 178845
2025.04.02 6800.0    0 6950.0 6950.0 6700.0 282054
2025.04.01 6800.0  2